In [1]:
import pandas as pd
import numpy as np

In [2]:
data_location = r"C:\Users\User\Documents\4. Fourth Year\Project\Railway-Fares\Data\\"
output_location = r"C:\Users\User\Documents\4. Fourth Year\Project\Railway-Fares\Data\Cleansed Data\\"

In [3]:
# Read in all files needed

nlc = pd.read_csv(data_location + "nlc_codes.csv", dtype = str)
postcodes = pd.read_csv(data_location + "Station Postcodes.csv", dtype = str)
distances = pd.read_csv(data_location + "distances.csv", dtype = str)
simd_lookup = pd.read_csv(data_location + "SIMD Postcode Lookup.csv", dtype = str)


clusters = pd.read_csv(data_location + r"Parsed Fare Data\Station_Cluster.csv", dtype = str)
fare = pd.read_csv(data_location + r"Parsed Fare Data\Fare.csv", dtype = str)
flow = pd.read_csv(data_location + r"Parsed Fare Data\Flow.csv", dtype = str)
ticket_types = pd.read_csv(data_location + r"Parsed Fare Data\Ticket_Types.csv", dtype = str)

In [4]:
# Join postcodes to nlc codes - we only scraped postcodes for Scottish stations so this defines the scope of stations

nlc_lookup = nlc.merge(postcodes, left_on = "Location", right_on = "Station", how = "right")

In [5]:
failed_nlc = nlc_lookup[nlc_lookup['NLC'].isna()]

In [6]:
# Get the successful joins from above and merge onto postcode lookup to get SIMD datazone

station_master = nlc_lookup[nlc_lookup['NLC'].notnull()][["NLC Clipped", "CRS", "Station", "Latitude", "Longitude", "Postcode"]]

station_master = station_master.merge(simd_lookup[["Postcode", "DZ"]], on = "Postcode", how = "left")

station_master = station_master.rename(columns = {"NLC Clipped": "NLC", "DZ": "Datazone"})

In [7]:
# Merge distances with station master to get distances with only NLC codes

nlc_dists = distances.merge(station_master, left_on = "Origin", right_on = "Station", suffixes = ("", "_origin"))
nlc_dists = nlc_dists.merge(station_master, left_on = "Destination", right_on = "Station", suffixes = ("", "_dest"))
nlc_dists = nlc_dists[["NLC", "NLC_dest", "Distance"]].rename(columns = {"NLC": "Origin", "NLC_dest": "Destination"})

In [8]:
flow = flow[["ORIGIN_CODE", "DESTINATION_CODE", "ROUTE_CODE", "DIRECTION", "END_DATE", "START_DATE", "FLOW_ID"]]
flow_inscope = flow.merge(station_master[["NLC", "Station"]], left_on = "ORIGIN_CODE", right_on = "NLC")
flow_inscope = flow_inscope.merge(station_master[["NLC", "Station"]], left_on = "DESTINATION_CODE", right_on = "NLC", suffixes = ("", "_dest"))

In [9]:
fares_out = fare.merge(flow_inscope, on = "FLOW_ID")
fares_out = fares_out[["TICKET_CODE", "FARE", "RESTRICTION_CODE", "ORIGIN_CODE", "DESTINATION_CODE", "DIRECTION", "END_DATE", "START_DATE"]]

In [10]:
ticket_types_inscope = ticket_types[(ticket_types["TKT_GROUP"] == 'S') & (ticket_types["TKT_TYPE"] == 'S')][["TICKET_CODE", "DESCRIPTION"]]

In [11]:
fares_inscope = fares_out.merge(ticket_types_inscope, on = "TICKET_CODE")[["TICKET_CODE", "FARE", "RESTRICTION_CODE", "ORIGIN_CODE", "DESTINATION_CODE", "DIRECTION", "END_DATE", "START_DATE"]]

In [12]:
# Output final files

station_master.to_csv(output_location + "station.csv", index = False)
nlc_dists.to_csv(output_location + "distances.csv", index = False)
fares_inscope.to_csv(output_location + "fares.csv", index = False)
ticket_types.to_csv(output_location + "ticket_types.csv", index = False)

In [30]:
simd = pd.read_csv(data_location + "simd2020_withinds.csv", dtype = str)

In [31]:
simd = simd.replace('*', np.nan)
for col in simd.columns:
    if '%' in simd[col][0]:
        print(col)
        simd[col] = simd[col].str.rstrip('%').astype(float) / 100

income_rate
employment_rate
DEPRESS
LBWT
Attendance
not_participating
University
overcrowded_rate
nocentralheating_rate
broadband


In [33]:
simd.merge(station_master, left_on = "Data_Zone", right_on = "Datazone")

,Data_Zone,Intermediate_Zone,Council_area,Total_population,Working_Age_population,SIMD2020v2_Rank,SIMD_2020v2_Percentile,SIMD2020v2_Vigintile,SIMD2020v2_Decile,SIMD2020v2_Quintile,...,PT_post,PT_retail,broadband,NLC,CRS,Station,Latitude,Longitude,Postcode,Datazone
0,S01006587,City Centre East,Aberdeen City,571,500,1562,23,5,3,2,...,6.699981447,6.170816327,0.30,8976,ABD,Aberdeen,57.1435,-2.0985,AB11 6FD,S01006587
1,S01006753,Dyce,Aberdeen City,868,511,5837,84,17,9,5,...,4.600481928,11.19353505,0.17,8905,DYC,Dyce,57.2057,-2.1925,AB21 7GL,S01006753
2,S01006801,Mearns and Laurencekirk,Aberdeenshire,991,539,5662,82,17,9,5,...,9.360572699,9.596338912,0.26,8931,LAU,Laurencekirk,56.836999999999996,-2.4652,AB30 1BE,S01006801
3,S01006832,Stonehaven North,Aberdeenshire,982,586,6262,90,18,9,5,...,5.90946793,5.243202138,0.01,8964,STN,Stonehaven,56.9666,-2.2255,AB39 2NE,S01006832
4,S01006851,Banchory-Devenick and Findon,Aberdeenshire,773,517,4304,62,13,7,4,...,4.993956522,6.776881988,0.00,8860,PLN,Portlethen,57.0606,-2.1286,AB12 4PT,S01006851
5,S01006907,Inverurie South,Aberdeenshire,712,498,4028,58,12,6,3,...,3.987683946,6.366170569,0.00,8924,INR,Inverurie,57.2864,-2.3741,AB51 4TN,S01006907
6,S01007001,"Insch, Oyne and Ythanwells",Aberdeenshire,1062,601,4811,69,14,7,4,...,4.721702746,5.537753432,0.01,8922,INS,Insch,57.3374,-2.6172,AB52 6JT,S01007001
7,S01007009,Huntly,Aberdeenshire,1047,624,4587,66,14,7,4,...,8.076090819,7.574007897,0.04,8921,HNT,Huntly,57.4445,-2.7756,AB54 6HY,S01007009
8,S01007144,Monikie,Angus,873,503,4190,61,13,7,4,...,11.90795042,12.37140363,0.45,9020,BYL,Barry Links,56.4928,-2.7457,DD7 7RS,S01007144
9,S01007145,Monifieth West,Angus,863,435,5022,72,15,8,4,...,7.908520179,9.542393498,0.00,9013,BSI,Balmossie,56.4748,-2.8386,DD5 4QD,S01007145
